In [1]:
import sys
sys.path.append('../src/')

import config
import logging
import scanpy as sc
from refcm import RefCM

config.start_logging(logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ds = {
    'MTG': sc.read_h5ad('../data/ALM.h5ad'), 
    'ALM': sc.read_h5ad('../data/VISP.h5ad'), 
    'VISp': sc.read_h5ad('../data/MTG.h5ad')
}

In [4]:
q, ref = 'MTG', 'ALM'

rcm = RefCM()
m = rcm.annotate(ds[q], q, ds[ref], ref, 'labels34', 'labels34')
m.eval('labels34')
m.display_matching_costs('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
|████████████████| [100.00% ] : 02:34
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings


In [6]:
rcm = RefCM(discovery_threshold=0)

for i, q_id in enumerate(ds):
    for ref_id in [d for d in ds if d != q_id]:
        m = rcm.annotate(ds[q_id], q_id, ds[ref_id], ref_id, 'labels34', 'labels34')
        m.eval('labels34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
|████████████████| [100.00% ] : 00:38
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 00:43
[matchings       ] [INFO    ] : mapped MTG                  to VISp                
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 00:36
[matchings       ] [INFO    ] : mapped ALM                  to MTG                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings
|████████████████| [100.00% ] : 01:01
[matchings 